In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    input_index = np.array(range(0, length, 24))
    label_index = input_index + 24* (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), pred_day, labels

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters['watershed']

'han_auto_13'

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 20)

In [12]:
run_num = range(len(folder))
real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

#ddday = 31
#mmmonth = 12

#length = 1

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])
    #df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
#                               colum_idx[idx], interpolation=interpolation_option[idx],
#                              first_file_no=i, month=mmmonth, day=ddday)

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/han/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['가평_2013.xlsx', '가평_2014.xlsx', '가평_2015.xlsx', '가평_2016.xlsx', '가평_2017.xlsx', '가평_2018.xlsx', '가평_2019.xlsx', '가평_2020.xlsx'], ['서상_2013.xlsx', '서상_2014.xlsx', '서상_2015.xlsx', '서상_2016.xlsx', '서상_2017.xlsx', '서상_2018.xlsx', '서상_2019.xlsx', '서상_2020.xlsx'], ['의암호_2013.xlsx', '의암호_2014.xlsx', '의암호_2015.xlsx', '의암호_2016.xlsx', '의암호_2017.xlsx', '의암호_2018.xlsx', '의암호_2019.xlsx', '의암호_2020.xlsx']]
data/han/자동/가평_2013.xlsx
data/han/자동/가평_2014.xlsx
data/han/자동/가평_2015.xlsx
data/han/자동/가평_2016.xlsx
data/han/자동/가평_2017.xlsx
data/han/자동/가평_2018.xlsx
data/han/자동/가평_2019.xlsx
data/han/자동/가평_2020.xlsx
You must check this make_timeseries : time length
time range in files :  2013-01-01 00:00  ~  2020-12-31 23:00
data/han/자동/서상_2013.xlsx
data/han/자동/서상_2014.xlsx
data/han/자동/서상_2015.xlsx
data/han/자동/서상_2016.xlsx
data/han/자동/서상_2017.xlsx
data/han/자동/서상_2018.xlsx
data/han/자동/서상_2019.xlsx


1/1 [==============================] - 0s 20ms/step - gen_loss: 12.5046 - disc_loss: 0.3754 - rmse: 0.4463 - val_loss: 0.3913
Epoch 49/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 16.1058 - disc_loss: 0.3685 - rmse: 0.4470 - val_loss: 0.4343
Epoch 50/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 14.0558 - disc_loss: 0.3471 - rmse: 0.4716 - val_loss: 0.4583
Epoch 51/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 12.6961 - disc_loss: 0.3808 - rmse: 0.5026 - val_loss: 0.4855
Epoch 52/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.6743 - disc_loss: 0.3647 - rmse: 0.4378 - val_loss: 0.4365
Epoch 53/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 12.2109 - disc_loss: 0.3834 - rmse: 0.4749 - val_loss: 0.4427
Epoch 54/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 11.4017 - disc_loss: 0.3628 - rmse: 0.4981 - val_loss: 1.0032
Epoch 55/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 9.1419 - disc_loss: 0.3269 - rmse: 0.3887 - val_loss: 0.3454
Epoch 107/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 23.5804 - disc_loss: 0.2903 - rmse: 0.5283 - val_loss: 0.3988
Epoch 108/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 12.7809 - disc_loss: 0.2918 - rmse: 0.4282 - val_loss: 0.3682
Epoch 109/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.7918 - disc_loss: 0.2932 - rmse: 0.4254 - val_loss: 0.3289
Epoch 110/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.6277 - disc_loss: 0.3070 - rmse: 0.3095 - val_loss: 0.3768
Epoch 111/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.3393 - disc_loss: 0.2965 - rmse: 0.4459 - val_loss: 0.3599
Epoch 112/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 10.8154 - disc_loss: 0.3009 - rmse: 0.4236 - val_loss: 0.4268
Epoch 113/2000
1/1 [===========

1/1 [==============================] - 0s 20ms/step - gen_loss: 8.3936 - disc_loss: 0.2667 - rmse: 0.4064 - val_loss: 0.3201
Epoch 165/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.0320 - disc_loss: 0.2741 - rmse: 0.3790 - val_loss: 0.3379
Epoch 166/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.6106 - disc_loss: 0.2647 - rmse: 0.3434 - val_loss: 0.3909
Epoch 167/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.2205 - disc_loss: 0.2522 - rmse: 0.4571 - val_loss: 0.3193
Epoch 168/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.3572 - disc_loss: 0.2681 - rmse: 0.3302 - val_loss: 0.9176
Epoch 169/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.4327 - disc_loss: 0.2920 - rmse: 0.3424 - val_loss: 0.3824
Epoch 170/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 22.8814 - disc_loss: 0.2865 - rmse: 0.5478 - val_loss: 0.4108
Epoch 171/2000
1/1 [=============

1/1 [==============================] - 0s 20ms/step - gen_loss: 9.6857 - disc_loss: 0.2433 - rmse: 0.3906 - val_loss: 0.3314
Epoch 223/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 7.6606 - disc_loss: 0.2449 - rmse: 0.3245 - val_loss: 0.3834
Epoch 224/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.1873 - disc_loss: 0.2509 - rmse: 0.4503 - val_loss: 0.4218
Epoch 225/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.4498 - disc_loss: 0.2631 - rmse: 0.3195 - val_loss: 0.4028
Epoch 226/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 20.1104 - disc_loss: 0.2564 - rmse: 0.3344 - val_loss: 0.3818
Epoch 227/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 10.5920 - disc_loss: 0.2605 - rmse: 0.4243 - val_loss: 0.3748
Epoch 228/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.9893 - disc_loss: 0.2442 - rmse: 0.4037 - val_loss: 0.3573
Epoch 229/2000
1/1 [============

...

1/1 [==============================] - 0s 20ms/step - gen_loss: 4.8005 - disc_loss: 0.1811 - rmse: 0.3095 - val_loss: 0.2845
Epoch 861/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.2984 - disc_loss: 0.1833 - rmse: 0.3312 - val_loss: 0.3007
Epoch 862/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 5.1929 - disc_loss: 0.1762 - rmse: 0.3340 - val_loss: 0.3015
Epoch 863/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.4234 - disc_loss: 0.1931 - rmse: 0.2750 - val_loss: 0.2874
Epoch 864/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7521 - disc_loss: 0.1777 - rmse: 0.2748 - val_loss: 0.3508
Epoch 865/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.5207 - disc_loss: 0.1821 - rmse: 0.3165 - val_loss: 0.3162
Epoch 866/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.4646 - disc_loss: 0.1777 - rmse: 0.3335 - val_loss: 0.3006
Epoch 867/2000
1/1 [===============

1/1 [==============================] - 0s 21ms/step - gen_loss: 5.7540 - disc_loss: 0.1783 - rmse: 0.3475 - val_loss: 0.3566
Epoch 919/2000
1/1 [==============================] - 0s 427ms/step - gen_loss: 5.6150 - disc_loss: 0.1867 - rmse: 0.3287 - val_loss: 0.3521
Epoch 920/2000
1/1 [==============================] - 0s 22ms/step - gen_loss: 5.3731 - disc_loss: 0.1802 - rmse: 0.3468 - val_loss: 0.3327
Epoch 921/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.4187 - disc_loss: 0.1828 - rmse: 0.2993 - val_loss: 0.2774
Epoch 922/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7350 - disc_loss: 0.1868 - rmse: 0.2990 - val_loss: 0.2702
Epoch 923/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 6.4157 - disc_loss: 0.1792 - rmse: 0.4062 - val_loss: 0.3092
Epoch 924/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.1083 - disc_loss: 0.1647 - rmse: 0.2889 - val_loss: 0.3769
Epoch 925/2000
1/1 [==============

1/1 [==============================] - 0s 21ms/step - gen_loss: 4.8404 - disc_loss: 0.1810 - rmse: 0.3002 - val_loss: 0.2869
Epoch 977/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.3517 - disc_loss: 0.1767 - rmse: 0.3247 - val_loss: 0.3654
Epoch 978/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.0424 - disc_loss: 0.1781 - rmse: 0.3429 - val_loss: 0.3114
Epoch 979/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.7033 - disc_loss: 0.1820 - rmse: 0.3202 - val_loss: 0.3132
Epoch 980/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.4793 - disc_loss: 0.1828 - rmse: 0.3248 - val_loss: 0.3544
Epoch 981/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.4217 - disc_loss: 0.1815 - rmse: 0.3373 - val_loss: 0.3417
Epoch 982/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.9172 - disc_loss: 0.1914 - rmse: 0.3290 - val_loss: 0.3120
Epoch 983/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.4050 - disc_loss: 0.1676 - rmse: 0.3375 - val_loss: 0.3112
Epoch 1035/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.4603 - disc_loss: 0.1759 - rmse: 0.3126 - val_loss: 0.3494
Epoch 1036/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.4311 - disc_loss: 0.1761 - rmse: 0.2924 - val_loss: 0.3050
Epoch 1037/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.5710 - disc_loss: 0.1754 - rmse: 0.2942 - val_loss: 0.3090
Epoch 1038/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 16.6170 - disc_loss: 0.1887 - rmse: 0.3123 - val_loss: 0.3968
Epoch 1039/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.0562 - disc_loss: 0.1739 - rmse: 0.3574 - val_loss: 0.2805
Epoch 1040/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 5.7176 - disc_loss: 0.1835 - rmse: 0.3085 - val_loss: 0.3455
Epoch 1041/2000
1/1 [=======

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.1757 - disc_loss: 0.1878 - rmse: 0.2872 - val_loss: 0.3262
Epoch 1093/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 5.8384 - disc_loss: 0.1788 - rmse: 0.3798 - val_loss: 0.3466
Epoch 1094/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.1628 - disc_loss: 0.1806 - rmse: 0.3203 - val_loss: 0.3644
Epoch 1095/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.2491 - disc_loss: 0.1727 - rmse: 0.2861 - val_loss: 0.3447
Epoch 1096/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.9125 - disc_loss: 0.1542 - rmse: 0.2848 - val_loss: 0.3165
Epoch 1097/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.6419 - disc_loss: 0.1790 - rmse: 0.3197 - val_loss: 0.3101
Epoch 1098/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.6568 - disc_loss: 0.1844 - rmse: 0.3219 - val_loss: 0.2716
Epoch 1099/2000
1/1 [========

In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  6
out_num_features :  1


In [58]:
print(real_df_all.shape)
rnn_target_column = 'toc'
print(rnn_target_column)

#df_length = real_df_all.shape[0] - 3000
df_length = real_df_all.shape[0]

fake_df = real_df_all.iloc[:df_length,:]

(70128, 36)
toc


In [59]:
#train_df, val_df, test_df, test_df2 = dataset_slice(fake_df, 0.8, 0.1, 0.1)
#teg_check = 'train=8, val=1, test=1'

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

#val_df, test_df,train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'val=1, test=1, train=8'

#test_df, val_df, train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'test=1, val=1, train=8'

#val_df, train_df, test_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'val=1, train=8, test=1'

#test_df, train_df, val_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'test=1, train=8, val=1'

train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, test=1, val=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (49089, 36) val_df.shape :  (14026, 36) test_df.shape: (7013, 36)


In [60]:
#fake_df.shape[0]/8760

In [61]:
plt.figure()
plt.plot(real_df_all.iloc[:, out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [62]:
plt.figure()
plt.plot(val_df.iloc[:, out_features[0]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [63]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [64]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

target_col_idx :  4
out_num_features :  1


In [65]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}

In [66]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

model_path = "save/" + watershed + "models/" + file_parameters['watershed'] + '/'+  pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    #if not os.path.exists('save/' + watershed):
    #    os.makedirs('save/' + watershed)
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/han/models/han_auto_13/toc/


In [67]:
#"save/" + watershed + "models/" + file_parameters['watershed'] + '/' + pa[indices[target_col_idx]]
#model_path

In [68]:
rnn_epochs

20

## 모델 학습

In [ ]:
gru_model = model_gru(
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=50,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt")

Epoch 1/50
10/10 [==============================] - ETA: 0s - loss: 0.6662 - mean_absolute_error: 0.5461 - nse: 0.1570
Epoch 00001: val_nse improved from -inf to 0.14236, saving model to save/best_model.h5
10/10 [==============================] - 10s 998ms/step - loss: 0.6662 - mean_absolute_error: 0.5461 - nse: 0.1570 - val_loss: 0.2137 - val_mean_absolute_error: 0.3638 - val_nse: 0.1424
Epoch 2/50
10/10 [==============================] - ETA: 0s - loss: 0.6348 - mean_absolute_error: 0.4208 - nse: 0.4163
Epoch 00002: val_nse improved from 0.14236 to 0.43265, saving model to save/best_model.h5
10/10 [==============================] - 10s 962ms/step - loss: 0.6348 - mean_absolute_error: 0.4208 - nse: 0.4163 - val_loss: 0.1409 - val_mean_absolute_error: 0.2866 - val_nse: 0.4327
Epoch 3/50
10/10 [==============================] - ETA: 0s - loss: 0.3605 - mean_absolute_error: 0.3503 - nse: 0.5760
Epoch 00003: val_nse improved from 0.43265 to 0.57931, saving model to save/best_model.h5
10/1

Epoch 22/50
10/10 [==============================] - ETA: 0s - loss: 0.4286 - mean_absolute_error: 0.3083 - nse: 0.6117
Epoch 00022: val_nse did not improve from 0.65115
10/10 [==============================] - 10s 975ms/step - loss: 0.4286 - mean_absolute_error: 0.3083 - nse: 0.6117 - val_loss: 0.1213 - val_mean_absolute_error: 0.2602 - val_nse: 0.5109
Epoch 23/50
10/10 [==============================] - ETA: 0s - loss: 0.3831 - mean_absolute_error: 0.2922 - nse: 0.6199
Epoch 00023: val_nse did not improve from 0.65115
10/10 [==============================] - 10s 977ms/step - loss: 0.3831 - mean_absolute_error: 0.2922 - nse: 0.6199 - val_loss: 0.1472 - val_mean_absolute_error: 0.2914 - val_nse: 0.4049
Epoch 24/50
10/10 [==============================] - ETA: 0s - loss: 0.2616 - mean_absolute_error: 0.2924 - nse: 0.6905
Epoch 00024: val_nse did not improve from 0.65115
10/10 [==============================] - 10s 973ms/step - loss: 0.2616 - mean_absolute_error: 0.2924 - nse: 0.6905 - v

10/10 [==============================] - ETA: 0s - loss: 0.2398 - mean_absolute_error: 0.2702 - nse: 0.7267
Epoch 00045: val_nse did not improve from 0.65115
10/10 [==============================] - 10s 981ms/step - loss: 0.2398 - mean_absolute_error: 0.2702 - nse: 0.7267 - val_loss: 0.1506 - val_mean_absolute_error: 0.2967 - val_nse: 0.3912
Epoch 46/50
10/10 [==============================] - ETA: 0s - loss: 0.1428 - mean_absolute_error: 0.2469 - nse: 0.7822
Epoch 00046: val_nse did not improve from 0.65115
10/10 [==============================] - 10s 964ms/step - loss: 0.1428 - mean_absolute_error: 0.2469 - nse: 0.7822 - val_loss: 0.1499 - val_mean_absolute_error: 0.2920 - val_nse: 0.3945
Epoch 47/50
10/10 [==============================] - ETA: 0s - loss: 0.2322 - mean_absolute_error: 0.2668 - nse: 0.7523
Epoch 00047: val_nse did not improve from 0.65115
10/10 [==============================] - 10s 984ms/step - loss: 0.2322 - mean_absolute_error: 0.2668 - nse: 0.7523 - val_loss: 0.1

## core / window.py / 

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [29]:
#train_df.shape, val_df.shape, test_df.shape

In [30]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvbnfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************       (*_*)  
#                   SSSSSSSS               ******            ******              *****    *****            ~( (>
#        SSSS         S  SSS               ******            ******              *****    *****            / (          
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

NameError: name 'stop' is not defined

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=real_df_all, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
from core.models import GRUModel, compile

In [ ]:
model112 = GRUModel(rnn_out_steps, out_num_features)
model112.load_weights(model_path+"gru.ckpt")
compile(model112)


In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    #model=model112,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    model=model112,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
##.evaluate(multi_window.train)
#model112.evaluate(multi_window.val.repeat(-1), steps=100)

In [ ]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    #print(df.shape)
    #print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    

    predictions = model(inputs).numpy()
    
    print("input : ", inputs.shape, "labels : ", labels.shape, "predictions : ", predictions.shape)
    #print(predictions.shape)
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    print("                label_day.shape : ",label_day.shape, "pred_day.shape : ", pred_day.shape)
    
    #plt.figure()
    #input_index = np.array(range(0, length, 24))
    #label_index = input_index + 24* (7 + predict_day)
    #plt.plot(input_index, inputs_day[:, 0, :], label='input')
    #plt.plot(label_index, label_day[:, predict_day, :], label='label')
    #plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    #plt.legend()
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), inputs_day, label_day, pred_day
    

In [ ]:

x = np.arange(len(val_nse))
width = 0.35
plt.figure()
plt.title(watershed + '  ['+start_year+','+end_year+']  ' + rnn_target_column)
plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(x + width, val_nse.values(), 0.3, label='NSE')
plt.xticks(x,val_nse.keys(), rotation=0)
_ = plt.legend()
plt.show()
